In [3]:
import pandas as pd
import string
import re
# from tqdm import tqdm

In [4]:
def clean(data):
    data=data.lower()
    data = re.sub('\[.*?\]','',data)
    data = re.sub("\\W", " ", data)
    data = re.sub('https?://\S+|www\.\S+', '', data)
    data = re.sub('<.*?>+', '', data)
    data = re.sub('[%s]' % re.escape(string.punctuation), '', data)
    data = re.sub('\n', '', data)
    data = re.sub('\w*\d\w*', '', data)
    return data

In [5]:
politifact = pd.read_json(r'C:\Users\abolf\Downloads\fact_checks\data\politifact.json')
snopes = pd.read_json(r'C:\Users\abolf\Downloads\fact_checks\data\snopes.json')
poli_cols = politifact.columns.to_list()
snopes_col = snopes.columns.to_list()

In [6]:
temp = poli_cols+snopes_col
temp_set = set(temp)
data_cols = ['claim','doc','topic','label']
df = pd.concat([politifact[data_cols],snopes[data_cols]], ignore_index=True)
df = df.sample(frac=1, ignore_index=True)
df['body']=df['doc']+" "+df['doc']
df['body'] = df['body'].apply(clean)


In [7]:
# drop the nulls
df =df.dropna(subset=['doc','claim','label','body'])

In [8]:
df.head(5)

,claim,doc,topic,label,body
0,"If parents ""don't have health insurance that S...",In an appearance onMeet the Presswith a little...,"[National, Health Care, John McCain]",half-true,in an appearance onmeet the presswith a little...
1,A meme provides information exonerating U.S. P...,"On July 6, 2019, financier Jeffrey Epstein — w...",[Politics],Mixture,on july financier jeffrey epstein who in...
2,"""Ohioans can expect to pay up to 41 percent mo...","Ohio Lt. Gov. Mary Taylor, who also is the dir...","[Ohio, Health Care, Government Regulation, Mar...",half-true,ohio lt gov mary taylor who also is the dir...
3,"Says U.S. Rep. Stephen Fincher ""breaks earmark...","Back when he was a candidate, U.S. Rep. Stephe...","[Tennessee, Congress, Federal Budget, Tennesse...",barely-true,back when he was a candidate u s rep stephe...
4,"In the event of a hurricane, mobile phone user...",As Hurricane Florence advanced on the mainland...,[Technology],Mixture,as hurricane florence advanced on the mainland...


In [35]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Taking care of categorical data
le = LabelEncoder()
df['f_label']=le.fit_transform(df['label'])

x=df['body']

x_train, x_test, y_train, y_test = train_test_split(df['body'],df['f_label'],test_size = 0.20)#, random_state = 42)

In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfid_vectorizer = TfidfVectorizer()
final_x_train = tfid_vectorizer.fit_transform(x_train)
final_x_test = tfid_vectorizer.transform(x_test)

In [48]:
from sklearn.linear_model import LogisticRegression

logestic_r = LogisticRegression()
logestic_r.fit(final_x_train,y_train)

c:\Users\abolf\Downloads\fact_checks\newEnv\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [52]:
predicted_values = logestic_r.predict(final_x_test)

In [54]:
logestic_r.score(final_x_test,y_test)

0.5190582959641256